In [ ]:
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sys.path.append('../src/')
from Biologging_Toolkit.applications.Surface import Surface

#### Enter deployment ID and path

Finalized dataset will be saved or loaded from path

In [ ]:
depid = 'ml17_280a'
path = 'D:/individus_brut/individus/ml17_280a/'
wav_path = 'D:/individus_brut/individus/ml17_280a/raw/'

#### Run cells to get acoustic features

Time can be constructed from extisting time array or by entering the path to sens5 dataset.

In [ ]:
inst = Surface(depid, 
            path = path,
            wav_path = wav_path
            )

In [ ]:
inst.get_timestamps(from_raw = True)

In [ ]:
dives, nresp, durresp, iresp = inst.get_surface_wav()

In [ ]:
nresp =  np.array(nresp).astype(float)
nresp[nresp >= 100] = np.nan
durresp = np.array(durresp)
durresp[durresp > 500] = np.nan
iresp = np.array(iresp)
iresp[iresp > 4] = 4

In [ ]:
df = pd.read_csv('F:/individus_brut/individus/ml17_280a/ml17_280a_dive.csv')

In [ ]:
corr_df = pd.DataFrame({'wind':np.sqrt(df.u10**2+df.v10**2), 'iresp':iresp, 'durresp':durresp, 'nresp':nresp})

In [ ]:
corr_df.corr()

In [ ]:
sns.pairplot(corr_df, kind = 'kde')

In [ ]:
from scipy.signal import welch, find_peaks, butter, filtfilt, sosfilt


In [ ]:
params = butter(N=4, fs = inst.samplerate, Wn=5000, btype='highpass', analog=False, output = 'sos')
ac_filtered = sosfilt(params, data)
plt.plot(ac_filtered)
plt.plot(data)


In [ ]:
f,t,Sxx = sg.spectrogram(data, fs = inst.samplerate, nperseg = 1024)

In [ ]:
resp = sg.medfilt(np.log10(Sxx)[np.argmin(abs(f - 1300))], int(np.ceil(0.3*(1/(t[1]-t[0])))//2*2+1))
peaks, _ = sg.find_peaks(resp, prominence=1, distance = int(0.5*(1/(t[1]-t[0]))))
params = butter(N=4, fs = 1 / (t[1] - t[0]), Wn=0.1, btype='highpass', analog=False, output = 'sos')
filt_resp = sosfilt(params, resp)

In [ ]:
np.mean(np.diff(t[peaks]))

In [ ]:
35/513*18400

In [ ]:
'''fig, ax = plt.subplots(1,2, figsize = (18,8))'''
fig, ax = plt.subplots(figsize = (18,8))
peaks, _ = sg.find_peaks(np.log10(Sxx[100]), prominence = 2, distance = 0.3 / (t[1] - t[0]))
'''ax[0].plot(np.log10(Sxx[100]))
ax[0].scatter(peaks, np.log10(Sxx[100])[peaks])'''
ax.imshow(np.log10(Sxx), aspect = 'auto')
ax.scatter(peaks, np.ones(len(peaks))*100)

In [ ]:
inst.ds.close()